# Rubrica
Que el script no contenga errores
Que los resultados del script sean reproducibles, ie que puedan ser reproducibles en cualquier computadora
El orden del código
La lógica del código
La inferencia empleada para los parámetros utilizados, tales como thresholds para filtrado de matriz, elección de thresholds para selección de alguna banda específica (alfa, beta, delta, etc.)
Los métodos utilizados, ie, si para construir la matriz de conectividad se utilizó coherencia espectral, PLV o correlación, que haya una razón de ello
Los resultados mostrados, tales como mapas de calor, grafos 2D, grafos 3D, el mostrar comunidades, hubs, nodos proporcionales al grado, filtrado de matriz de
Conclusión del análisis, ie, al finalizar el análisis dar una conclusión que resuma los resultados encontrados con una interpretación de los mismos
Que el análisis haya sido aplicado a cada uno de los ensayos/archivos del dataset

Graciela Montserrat Rodríguez Rodríguez
Proyecto, Modelos Computacionales I

Paper:An EEG Dataset of Neural 
Signatures in a Competitive 
two-Player Game Encouraging 
Deceptive Behavior

In [81]:
import numpy as np
import pandas as pd
import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io as sp
from scipy.io import loadmat
import mne
import glob
import os

In [90]:
#Abrir datos
ruta = r"C:\Datasets\Proyecto_Mon\Proyecto\Preprocessed\DecisionMaking\*.mat"
mat_files = {}

for archivo in glob.glob(ruta):
    nombre = os.path.basename(archivo)
    contenido = loadmat(archivo)
    mat_files[nombre] = contenido


In [86]:
#Codigo para analizar los archivos
if mat_files: # Solo si se cargaron archivos
    first_file_name = list(mat_files.keys())[0]
    data = mat_files[first_file_name] 

    print(f"\n--- Explorando '{first_file_name}' ---")
    print("Nivel 1 (contenido del archivo):")
    print(f"  Claves: {data.keys()}")
    if 'observer' in data:
        observer_content = data['observer']
        print(f"\nNivel 2 (contenido de 'observer'):")
        print(f"  Tipo: {type(observer_content)}")
        print(f"  Shape: {observer_content.shape}")

   ¿
        if observer_content.shape == (1, 1):
            level3_observer = observer_content[0, 0]
            print(f"\nNivel 3 (contenido de 'observer[0,0]'):")
            if hasattr(level3_observer, 'dtype') and level3_observer.dtype.names:
                print(f"  Campos/Claves: {level3_observer.dtype.names}")

                if level3_observer.dtype.names:
                    first_field_name = level3_observer.dtype.names[0]
                    level4_observer = level3_observer[first_field_name]
                    print(f"\nNivel 4 (contenido de '{first_field_name}' dentro de observer):")
                    print(f"  Tipo: {type(level4_observer)}")
                    if hasattr(level4_observer, 'shape'):
                        print(f"  Shape: {level4_observer.shape}")
                     
                    else:
                        print(f"  Contenido: {level4_observer}")
            else:
                print(f"  No hay campos nombrados. Tipo: {type(level3_observer)}, Shape: {level3_observer.shape if hasattr(level3_observer, 'shape') else 'N/A'}")
        else:
            print(f"  No es una estructura 1x1, Shape: {observer_content.shape}")
    else:
        print("La clave 'observer' no se encontró en el archivo.")




--- Explorando 'Player_sub01_Observer_sub02.mat' ---
Nivel 1 (contenido del archivo):
  Claves: dict_keys(['__header__', '__version__', '__globals__', 'observer', 'player'])

Nivel 2 (contenido de 'observer'):
  Tipo: <class 'numpy.ndarray'>
  Shape: (1, 1)

Nivel 3 (contenido de 'observer[0,0]'):
  Campos/Claves: ('fs', 'x', 'clab', 'y', 'className', 't', 'filename')

Nivel 4 (contenido de 'fs' dentro de observer):
  Tipo: <class 'numpy.ndarray'>
  Shape: (1, 1)


In [91]:
#Código para inspeccionar los campos, es decir, encontrar en donde estan los eeg

if mat_files:
    first_file_name = list(mat_files.keys())[0]
    first_file_content = mat_files[first_file_name]

    print(f"\n--- INSPECCIÓN DETALLADA DE '{first_file_name}' ---")

   
    expected_fields = ('fs', 'x', 'clab', 'y', 'className', 't', 'filename')

    print("\n##### Contenido de 'observer' #####")
    if 'observer' in first_file_content and first_file_content['observer'].shape == (1, 1):
        actual_observer_content = first_file_content['observer'][0, 0]

        if hasattr(actual_observer_content, 'dtype') and actual_observer_content.dtype.names:
            print(f"Campos disponibles en 'observer[0,0]': {actual_observer_content.dtype.names}")

            for field_name in expected_fields:
                if field_name in actual_observer_content.dtype.names:
                    field_value = actual_observer_content[field_name]
                    print(f"  --- Campo: '{field_name}' ---")
                    print(f"    Tipo: {type(field_value)}")
                    if hasattr(field_value, 'shape'):
                        print(f"    Shape: {field_value.shape}")
                    
                    if field_name == 'clab' and hasattr(field_value, 'flatten'):
                        
                        print(f"    Primeros 3 elementos (ejemplo): {field_value.flatten()[:3]}")
                    elif field_value is not None:
                        
                        if np.isscalar(field_value) or (isinstance(field_value, np.ndarray) and field_value.size == 1):
                             print(f"    Valor: {field_value}")
                        elif isinstance(field_value, np.ndarray) and field_value.size > 1:
                            
                            print(f"    Primeros 3 elementos (ejemplo): {field_value.flatten()[:3]}")
                else:
                    print(f"  --- Campo: '{field_name}' (No encontrado en observer[0,0]) ---")
        else:
            print(f"Contenido de 'observer[0,0]' no es una estructura con nombres de campo. Tipo: {type(actual_observer_content)}")
    else:
        print("'observer' no encontrado o no es una estructura (1,1).")


    print("\n##### Contenido de 'player' #####")
    if 'player' in first_file_content and first_file_content['player'].shape == (1, 1):
        actual_player_content = first_file_content['player'][0, 0]

        if hasattr(actual_player_content, 'dtype') and actual_player_content.dtype.names:
            print(f"Campos disponibles en 'player[0,0]': {actual_player_content.dtype.names}")

            for field_name in expected_fields:
                if field_name in actual_player_content.dtype.names:
                    field_value = actual_player_content[field_name]
                    print(f"  --- Campo: '{field_name}' ---")
                    print(f"    Tipo: {type(field_value)}")
                    if hasattr(field_value, 'shape'):
                        print(f"    Shape: {field_value.shape}")
                    if field_name == 'clab' and hasattr(field_value, 'flatten'):
                        print(f"    Primeros 3 elementos (ejemplo): {field_value.flatten()[:3]}")
                    elif field_value is not None:
                        if np.isscalar(field_value) or (isinstance(field_value, np.ndarray) and field_value.size == 1):
                             print(f"    Valor: {field_value}")
                        elif isinstance(field_value, np.ndarray) and field_value.size > 1:
                            print(f"    Primeros 3 elementos (ejemplo): {field_value.flatten()[:3]}")
                else:
                    print(f"  --- Campo: '{field_name}' (No encontrado en player[0,0]) ---")
        else:
            print(f"Contenido de 'player[0,0]' no es una estructura con nombres de campo. Tipo: {type(actual_player_content)}")
    else:
        print("'player' no encontrado o no es una estructura (1,1).")


--- INSPECCIÓN DETALLADA DE 'Player_sub01_Observer_sub02.mat' ---

##### Contenido de 'observer' #####
Campos disponibles en 'observer[0,0]': ('fs', 'x', 'clab', 'y', 'className', 't', 'filename')
  --- Campo: 'fs' ---
    Tipo: <class 'numpy.ndarray'>
    Shape: (1, 1)
    Valor: [[100]]
  --- Campo: 'x' ---
    Tipo: <class 'numpy.ndarray'>
    Shape: (350, 30, 484)
    Primeros 3 elementos (ejemplo): [11.73809814 -4.60339928  4.1760745 ]
  --- Campo: 'clab' ---
    Tipo: <class 'numpy.ndarray'>
    Shape: (1, 30)
    Primeros 3 elementos (ejemplo): [array(['F7'], dtype='<U2') array(['F3'], dtype='<U2')
 array(['Fz'], dtype='<U2')]
  --- Campo: 'y' ---
    Tipo: <class 'numpy.ndarray'>
    Shape: (4, 484)
    Primeros 3 elementos (ejemplo): [0 0 1]
  --- Campo: 'className' ---
    Tipo: <class 'numpy.ndarray'>
    Shape: (1, 4)
    Primeros 3 elementos (ejemplo): [array(['sponL'], dtype='<U5') array(['sponT'], dtype='<U5')
 array(['instL'], dtype='<U5')]
  --- Campo: 't' ---
    Tip

In [ ]:
#Matrices de correlación
correlation_matrices_observer = {}
correlation_matrices_player = {}


if mat_files:
    print("\nMatrices de correlación del jugador y su respectivo jugador...")
    for file_name, file_content in mat_files.items():
        try:
            observer_data_struct = file_content['observer'][0, 0]
            eeg_observer_data_3d = observer_data_struct['x'] #(puntos_de_tiempo, canales, épocas)

            # Reestructurar los datos para hacerla de 2 dimensiones y usar np.corrcoef: (canales x (puntos_de_tiempo * épocas))
            eeg_observer_reshaped = np.transpose(eeg_observer_data_3d, (1, 0, 2))
            # Aplanar los puntos en el tiempo y épocas en una sola dimensión para las muestras
            num_channels_obs = eeg_observer_reshaped.shape[0]
            num_samples_per_channel_obs = eeg_observer_reshaped.shape[1] * eeg_observer_reshaped.shape[2]
            eeg_observer_2d = eeg_observer_reshaped.reshape(num_channels_obs, num_samples_per_channel_obs)

            # Calcular la matriz de correlación para el observador
            corr_matrix_observer = np.corrcoef(eeg_observer_2d)
            correlation_matrices_observer[file_name] = corr_matrix_observer

            # PLAYER
            player_data_struct = file_content['player'][0, 0]
            eeg_player_data_3d = player_data_struct['x'] 

           
            eeg_player_reshaped = np.transpose(eeg_player_data_3d, (1, 0, 2))
            num_channels_play = eeg_player_reshaped.shape[0]
            num_samples_per_channel_play = eeg_player_reshaped.shape[1] * eeg_player_reshaped.shape[2]
            eeg_player_2d = eeg_player_reshaped.reshape(num_channels_play, num_samples_per_channel_play)

           
            corr_matrix_player = np.corrcoef(eeg_player_2d)
            correlation_matrices_player[file_name] = corr_matrix_player

            print(f"Calculadas matrices de correlación para '{file_name}' (observador y jugador). "
                  f"Shape de datos EEG observador (original 3D): {eeg_observer_data_3d.shape}, "
                  f"Shape 2D para correlación: {eeg_observer_2d.shape}")

        except KeyError as e:
            print(f"Error: La clave {e} no se encontró en el archivo {file_name}. ¿Estructura inesperada?")
        except Exception as e:
            print(f"Ocurrió un error inesperado al procesar {file_name}: {e}")

    print("\n--- ¡Matrices de correlación calculadas para todos los archivos! ---")

In [135]:
#   #Data Frame por cada observador en decision making
# print("\n##### VISUALIZACIÓN DE TODAS LAS MATRICES DE CORRELACIÓN #####")

#     # Mostrar DataFrames para el OBSERVADOR
#     print("\n--- Matrices de Correlación para OBSERVADORES ---")
#     for file_name, corr_matrix in correlation_matrices_observer.items():
#         print(f"\n**{file_name} (OBSERVADOR)**") 
#         df_obs = pd.DataFrame(corr_matrix)
#         if channel_labels:
#             df_obs.columns = channel_labels
#             df_obs.index = channel_labels
#         display(df_obs) 

#     # Mostrar DataFrames para el PLAYER
#     print("\n--- Matrices de Correlación para PLAYERS ---")
#     for file_name, corr_matrix in correlation_matrices_player.items():
#         print(f"\n**{file_name} (PLAYER)**") # Encabezado para cada DataFrame
#         df_play = pd.DataFrame(corr_matrix)
#         if channel_labels:
#             df_play.columns = channel_labels
#             df_play.index = channel_labels
#         display(df_play) 

In [137]:
#Umbralización, eliminación de la diágonal y creación de grafos del observador
for key, matrix_data in correlation_matrices_observer.items():
    np.fill_diagonal(matrix_data, 0) # Eliminar diagonales
    mean = np.mean(matrix_data[matrix_data != 0]) # Solo elementos no diagonales
    std = np.std(matrix_data[matrix_data != 0])   # Solo elementos no diagonales
    threshold = mean + std
    print(f"Matrix: {key}, Threshold: {threshold:.4f}")
    matrix_data[matrix_data < threshold] = 0

for key, corr_matrix in correlation_matrices_observer.items():
        print(f"\n**{file_name} (OBSERVADOR)**") 
        df_obs = pd.DataFrame(corr_matrix)
        if channel_labels:
            df_obs.columns = channel_labels
            df_obs.index = channel_labels
        display(df_obs) 
Grafos= {}
for key, corr_matrix in correlation_matrices_observer.items():
    G = nx.from_numpy_array(corr_matrix)
    Grafos[key] = G


    

Matrix: Player_sub01_Observer_sub02.mat, Threshold: 0.9067
Matrix: Player_sub03_Observer_sub06.mat, Threshold: 0.8988
Matrix: Player_sub04_Observer_sub05.mat, Threshold: 0.9940
Matrix: Player_sub05_Observer_sub04.mat, Threshold: 0.9174
Matrix: Player_sub06_Observer_sub03.mat, Threshold: 0.9720
Matrix: Player_sub07_Observer_sub08.mat, Threshold: 0.9400
Matrix: Player_sub08_Observer_sub07.mat, Threshold: 0.9887
Matrix: Player_sub09_Observer_sub10.mat, Threshold: 0.9711
Matrix: Player_sub10_Observer_sub09.mat, Threshold: 0.9983
Matrix: Player_sub11_Observer_sub12.mat, Threshold: 0.8993
Matrix: Player_sub12_Observer_sub11.mat, Threshold: 1.0056
Matrix: Player_sub13_Observer_sub14.mat, Threshold: 0.9857
Matrix: Player_sub14_Observer_sub13.mat, Threshold: 0.9877
Matrix: Player_sub15_Observer_sub16.mat, Threshold: 0.9267
Matrix: Player_sub16_Observer_sub15.mat, Threshold: 0.9124
Matrix: Player_sub17_Observer_sub18.mat, Threshold: 0.8444
Matrix: Player_sub18_Observer_sub17.mat, Threshold: 0.93

,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,...,Pz,P4,P8,PO3,PO4,O1,O2,F9,F10,Fp2
F7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FC5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FC1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FC2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FC6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



**Player_sub24_Observer_sub23.mat (OBSERVADOR)**


,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,...,Pz,P4,P8,PO3,PO4,O1,O2,F9,F10,Fp2
F7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
F3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
Fz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
F4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
F8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
FC5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
FC1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
FC2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
FC6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
T7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0



**Player_sub24_Observer_sub23.mat (OBSERVADOR)**


...

,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,...,Pz,P4,P8,PO3,PO4,O1,O2,F9,F10,Fp2
F7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
F3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
Fz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
F4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
F8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
FC5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
FC1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
FC2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
FC6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
T7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0



**Player_sub24_Observer_sub23.mat (OBSERVADOR)**


,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,...,Pz,P4,P8,PO3,PO4,O1,O2,F9,F10,Fp2
F7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FC5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FC1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FC2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FC6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



**Player_sub24_Observer_sub23.mat (OBSERVADOR)**


,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,...,Pz,P4,P8,PO3,PO4,O1,O2,F9,F10,Fp2
F7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FC5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FC1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FC2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FC6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
##Tipo de grafo: nodos, aristas, conexiones
##Matrices de adyacencia, por condiciones
## En el escrito deben estar los grafos. 3D o 2D, no importa mucho para Oswaldo, para fer  tal vez sí iji
## Metricas: grado, ¿qué tipo de grafo es?, binarizar, y sacar hubs.
## Para sacar umbral: media+desviación estandar
## Coeficiente de cluster y rutas más cortas, apartir de ahí mundo pequeño
## Betweeness, intermediación, cercanía, rich club, comunidades, modularidad, gráficas de escala libre (es decir, que la distribución de grados siga 
## una ley potencia)
## A partir de ahí, hacer gráficas y estadísticas adecuadas, histograma y percentiles

#Bibliografia
Nayak, C. S., & Anilkumar, A. C. (2025, 3 agosto). EEG Normal waveforms. StatPearls - NCBI Bookshelf. https://www.ncbi.nlm.nih.gov/books/NBK539805/